# Standards

Pyatoa enforces a few standards for book keeping. Many of these standards are encapsulated in the functions found in :module:`~pyatoa.utils.form`.

---

## Receiver codes

Receivers can be identified multiple ways in the workflow. Most of the time when Pyatoa asks the user to identify a receiver, it will be through its __code__:

*  __CODE__: The full SEED standard code __NN.SSS.LL.CCC__ (where __N__=Network, __S__=Station, __L__=Location, __C__=Channel); e.g. [NZ.BFZ.10.HHZ](https://www.geonet.org.nz/data/network/sensor/BFZ)  
> **_wildcards_**: 
Wildcards are accepted and encouraged when specifying full codes. For example, in data gathering, selecting all available components is accomplished by specifying code NZ.BFZ.10.HH?, where the ? character is a single-valued wildcard.
*  __NET / NETWORK__: Two character network code __NN__; e.g. NZ
*  __STA / STATION__: Multi (3 or 4) character station code __SSS__: e.g. BFZ
*  __LOC / LOCATION__: Two digit location code, for receiver sites that have multiple instruments (e.g. borehole and surface broadbands). e.g. 10
> **_unknown location code_**: 
In my experience, location naming is not standard across data services. It is usually acceptable to provide wildcards to avoid having to determine the specific location code, e.g. NZ.BFZ.??.HH?
*  __CHA / CHANNEL__: Three character channel code __BIO__ (where __B__=Band code, __I__=Instrument code, __O__= Orientation code), e.g. HHZ
> **_seed convention_**: 
When saving new data, band code is enforced via the data sampling rate, following the format of the [SEED convention](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/).

*  __COMP / COMPONENT__: One character channel component, a.k.a the orientation code in __CHANNEL__
> **_expected components_**: 
In seismology __COMPONENT__ is usually one of the following N: North, E: East, Z: Vertical, R: Radial, T: Transverse

___

## Event ID

The natural separation point for data objects is per event. Said differently, processing in Pyatoa happens independently, one event at a time. Unique event ID's are __critical__ for book keeping functionality within Pyatoa. Since Pyatoa was developed in New Zealand, the standards reflects the GeoNet Event ID formatting. 

> **_STANDARD_**: 

*  Event IDs are generated from the resource_id of the ObsPy Event objects that are used to define event parameters.
*  Event IDs must be strings and all event ids within an inversion must be unique. 
*  Each ASDFDataSet must correspond to a single event, and be saved using the associated event id.

> **_SUPPORTED SERVICES_**:
*  [GeoNet](https://quakesearch.geonet.org.nz/)
*  IRIS
*  .ndk files from SPUD, GCMT
*  USGS Events, and associated ANSS ComCat moment tensors

> **_NOTE_**:
If unique event identifiers are required, you can monkey patch the function :func:`~pyatoa.utils.form.format_event_name`

___

## Iteration

Iteration number is used to keep track of the number of full iterations (one iteration more or less contains a forward simulation, adjoint simulation, and line search). The :class:`~pyatoa.core.config.Config` class keeps track of the iteration as an internal attribute, which is used for saving waveforms, auxiliary data, figures, etc.

> **_STANDARD_**: 
*  Iteration must be an integer that starts at 1. Iterations can not be negative.
*  The `iteration tag` is a string formatted version of the iteration used for saving data, and is formatted e.g. __i01__ for iteration 1.

___

## Step Count

Within a single iteration, multiple fuction evaluations take place, relating to the intial model assessment (step 0), and each subsequent line search function evaluation, each which try to reduce the objective function via some step length of model perturbations (step count > 0). As with iterations, the Config class keeps track of step count information for internal tagging.

> **_STANDARD_**: 
*  Step count must be an integer that starts from 0. Step counts can not be negative.
*  The `step tag` is a string formatted version of the iteration used for saving data, and is formatted e.g. __s00__ for step count 0.

___

## Observation Waveforms

Observation data that will not be gathered from online web-services, e.g. temporary network data that is locally stored, must be available using a specific directory and file naming format that mimics data center convention. This is reflected in the function :func:`~pyatoa.core.gatherer.Fetcher.fetch_obs_by_dir`.

> **_STANDARD_**:
*  Default Directory Template: path/to/observed/YYYY/NN/SSS/CCC/  
*  Default File ID Template: NN.SSS.LL.CCC.YYYY.DDD  
Where:
    * YYYY: The year with the century (e.g., 1987)  
    * NN: The network code (e.g. NZ)  
    * SSS: The station code (e.g. BFZ)  
    * LL: The location code (e.g. 10)  
    * CCC: The channel code (e.g. HHZ.D)  
    * DDD: The julian day of the year (January 1 is 001)  
    
Example directory for station NZ.BFZ, for the day 2018-02-18: **path/to/observed/2018/NZ/BFZ/HHZ/NZ.BFZ.10.HHZ.D.2018.049**

___

## Station Response

As with observation data, response files that are stored locally on disk (a.k.a StationXML files, dataless files) must be saved to a specific directory and with specific file naming. These will be searched for by the function :func:`pyatoa.core.gatherer.Fetcher.fetch_resp_by_dir`

> **_STANDARD_**:
* Default Directory Template: path/to/responses/__SSS.NN__
* Default File ID Template: __RESP.NN.SSS.LL.CCC__  
Where:
    * NN: The network code (e.g. NZ)  
    * SSS: The station code (e.g. BFZ)  
    * LL: The location code (e.g. 10)  
    * CCC: The channel code (e.g. HHZ.D)  

Example directory for station NZ.BFZ: **path/to/response/BFZ.NZ/RESP.NZ.BFZ.10.HHZ**

___

## Synthetic Waveforms

Currently Pyatoa is written to work with ASCII synthetic seismograms outputted by SPECFEM3D. Synthetic waveforms therefore must follow the given naming convention set by SPECFEM3D.

> **_STANDARD_**:
* Synthetic waveforms must saved in the form: __NN.SSS.BIO.__sem**U**  
(where __N__=Network, __S__=Station, __B__=Band code, __I__=Instrument code, __O__=Orientation code, __U__=Unit code)
* Instrument code __I__ must __always__ be the letter __X__, following SEED convention for naming synthetic waveforms. 
* Unit code is dictated by the chosen output units in the SPECFEM3D Par_file, and correspond to: __d__=displacement, __v__=velocity, __a__=acceleration.

Example for displacement synthetic waveforms for the vertical component of New Zealand station BFZ: **NZ.BFZ.BXZ**